In [1]:
import numpy as np
import pandas as pd
import requests
import math
import xlsxwriter
from scipy import stats
from secret import IEX_CLOUD_API_TOKEN


In [2]:
stock_data = pd.read_csv('sp_500_stocks.csv')
stock_data

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


In [3]:
base_url = 'https://cloud.iexapis.com/'
symbol = 'AAPL'
api_url = f'{base_url}stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
print(data)


{'companyName': 'Apple Inc', 'marketcap': 2613943698660, 'week52high': 175.37, 'week52low': 123.98, 'week52highSplitAdjustOnly': 176.15, 'week52lowSplitAdjustOnly': 124.17, 'week52change': 0.006840248745182409, 'sharesOutstanding': 15821946000, 'float': 0, 'avg10Volume': 52665258, 'avg30Volume': 56925376, 'day200MovingAvg': 146.13, 'day50MovingAvg': 156.87, 'employees': 164000, 'ttmEPS': 5.88, 'ttmDividendRate': 0.9170023129162208, 'dividendYield': 0.005550525470106051, 'nextDividendDate': '', 'exDividendDate': '2023-02-10', 'nextEarningsDate': '2023-05-04', 'peRatio': 27.46575846276702, 'beta': 1.2982257376011077, 'maxChangePercent': 63.14427706165554, 'year5ChangePercent': 2.947632519641389, 'year2ChangePercent': 0.24783605695588729, 'year1ChangePercent': 0.006840248745182409, 'ytdChangePercent': 0.27347244586720776, 'month6ChangePercent': 0.19920474020134105, 'month3ChangePercent': 0.2278287029949182, 'month1ChangePercent': 0.06005774783445639, 'day30ChangePercent': 0.06587096774193

In [4]:
data['year1ChangePercent']
Symbol = stock_data['Ticker']
Symbol

0         A
1       AAL
2       AAP
3      AAPL
4      ABBV
       ... 
500     YUM
501     ZBH
502    ZBRA
503    ZION
504     ZTS
Name: Ticker, Length: 505, dtype: object

In [5]:
def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i+n]
symbol_strings = []

symbol_groups = list(chunks(Symbol, 100))

for i in range(0, len(symbol_groups)):
    symbol_strings.append(','. join(symbol_groups[i]))
my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']
my_columns
# symbol_string

['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

In [6]:
df = pd.DataFrame(columns=my_columns)
df

,Ticker,Price,One-Year Price Return,Number of Shares to Buy


In [12]:

# final_dataframe = pd.DataFrame(columns = my_columns)
# final_dataframe
tickers = []
prices = []
change_percent = []
no_of_shares_to_buy = []
for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'{base_url}stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        try:
            current_symbol = symbol
            symbol_price = data[symbol]['quote']['latestPrice']
            symbol_change = data[symbol]['stats']['year1ChangePercent']
            shares_to_buy = 'N/A'
        except:
            pass
        else:
            tickers.append(current_symbol)
            change_percent.append(symbol_change)
            prices.append(symbol_price)
            no_of_shares_to_buy.append(shares_to_buy)
output = {
    'Ticker': tickers,
    'Price': prices,
    'One-Year Price Return': change_percent,
    'Number of Shares to Buy': no_of_shares_to_buy
}
final_dataframe = pd.DataFrame(output)



In [15]:
final_dataframe.sort_values('One-Year Price Return', inplace=True, ascending=False)
final_dataframe.reset_index(drop=True)
final_dataframe = final_dataframe[:50]

In [16]:
portfolio_size = input('Enter the size of your portfolio ')

50